In [ ]:
from datetime import date, timedelta
import geopandas as gpd
import matplotlib as mpl
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib.lines import Line2D
import numpy as np
import pandas as pd
from pylr2 import regress2
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import udf, col
import statsmodels.api as sm
import statsmodels.formula.api as smf
import rioxarray as rxr

In [ ]:
from src import constants
from src.data import spark_postgis
from src.utils import raster_utils

## 0. Setup

In [ ]:
spark = spark_postgis.get_spark()

In [ ]:
# Occasionally, we get sets of three shots with a disturbance between.
# Sometimes it is valid to count these as separate samples
# (e.g. s1a -- disturbance -- s1b -- s2,
# where the pair s1a-s2 is a treatment sample and s1b-s2 is a control sample).
# But other times, it's really two measurements of the same sample
# (e.g. s1a -- s1b -- disturbance -- s2, where s1a-s2 and s1b-s2 are both
# measurements of the same disturbance event).
# Just to be on the safe side, we can remove all the duplicates.
# This function should be run on the control and treatment sets separately.

def remove_duplicates(df):
    print(
        "Found {} s1 duplicates".format(
            len(df[df.duplicated(subset=["t1_shot_number"])])
        )
    )
    print(
        "Found {} s2 duplicates".format(
            len(df[df.duplicated(subset=["t2_shot_number"])])
        )
    )
    df = df.drop_duplicates(subset=["t1_shot_number"], keep="first")
    df = df.drop_duplicates(subset=["t2_shot_number"], keep="first")

    return df

In [ ]:
degrade_sdf = spark.read.parquet(
    (constants.RESULTS_PATH / "gedi_degradation_glad_0d_l24a").as_posix()
)

@udf(returnType=IntegerType())
def get_days(time_delta):
    return time_delta.days


degrade_sdf = degrade_sdf.withColumn(
    "time_diff",
    (degrade_sdf["t2_absolute_time"] - degrade_sdf["t1_absolute_time"]),
)
degrade_sdf = degrade_sdf.withColumn("time_diff", get_days(col("time_diff")))
glad_df = gpd.GeoDataFrame(degrade_sdf.toPandas(), geometry="t2_geom").copy()
glad_df.loc[glad_df.control_disturbance > 0, "sample_grp"] = "control"
# Note: points may have a control disturbance as well as a measured disturbance.
# in that case, we include them in the treatment group; we don't care that they
# were also disturbed at another, unmeasured time.
glad_df.loc[glad_df.measured_disturbance > 0, "sample_grp"] = "treatment"
print(len(glad_df))
print(len(glad_df[glad_df["sample_grp"] == "treatment"]))
print(len(glad_df[glad_df["sample_grp"] == "control"]))
control_df = remove_duplicates(glad_df[glad_df["sample_grp"] == "control"])
control_df["sample_grp"] = "control"
treatment_df = remove_duplicates(glad_df[glad_df["sample_grp"] == "treatment"])
treatment_df["sample_grp"] = "treatment"
glad_df = pd.concat([control_df, treatment_df])
print(len(glad_df))
print(len(glad_df[glad_df["sample_grp"] == "treatment"]))
print(len(glad_df[glad_df["sample_grp"] == "control"]))
control_n = len(glad_df[glad_df["sample_grp"] == "control"])

In [ ]:
gpd.GeoDataFrame(glad_df[glad_df.sample_grp == "treatment"]).drop(columns=["t1_geom"]).to_feather(
    constants.RESULTS_PATH / "glad_treatment_0d.feather"
)

In [ ]:
degrade_sdf = spark.read.parquet(
    (constants.RESULTS_PATH / "gedi_degradation_afc_2022").as_posix()
)

@udf(returnType=IntegerType())
def get_days(time_delta):
    return time_delta.days

degrade_sdf = degrade_sdf.withColumn(
    "time_diff",
    (degrade_sdf["t2_absolute_time"] - degrade_sdf["t1_absolute_time"]),
)
degrade_sdf = degrade_sdf.withColumn("time_diff", get_days(col("time_diff")))
afc_df = gpd.GeoDataFrame(degrade_sdf.toPandas(), geometry="t2_geom").copy()
afc_df.loc[afc_df.control_disturbance > 0, "sample_grp"] = "control"
# Note: points may have a control disturbance as well as a measured disturbance.
# in that case, we include them in the treatment group; we don't care that they
# were also disturbed at another, unmeasured time.
afc_df.loc[afc_df.measured_disturbance > 0, "sample_grp"] = "treatment"
print(len(afc_df))
print(len(afc_df[afc_df["sample_grp"] == "treatment"]))
print(len(afc_df[afc_df["sample_grp"] == "control"]))
control_df = remove_duplicates(afc_df[afc_df["sample_grp"] == "control"])
control_df["sample_grp"] = "control"
treatment_df = remove_duplicates(afc_df[afc_df["sample_grp"] == "treatment"])
treatment_df["sample_grp"] = "treatment"
afc_df = pd.concat([control_df, treatment_df])
print(len(afc_df))
print(len(afc_df[afc_df["sample_grp"] == "treatment"]))
print(len(afc_df[afc_df["sample_grp"] == "control"]))

In [ ]:
def temporal_adjust(df_orig):
    bins = np.arange(0, 1200, 60)
    df_orig["time_diff_bin"] = pd.cut(df_orig["time_diff"], bins=bins)
    treatment_dist = df_orig[df_orig.sample_grp == "treatment"].groupby("time_diff_bin").size().reset_index(name="count")

    new_control = []
    for i in range(0, len(treatment_dist)):
        bin, count = treatment_dist.iloc[i]["time_diff_bin"], treatment_dist.iloc[i]["count"]
        d = df_orig[(df_orig.sample_grp == "control") & (df_orig.time_diff_bin == bin)]
        if len(d) < count:
            count = len(d)
        new_control.append(d.sample(count, replace=False))

    new_control = pd.concat(new_control)
    return pd.concat([new_control, df_orig[df_orig.sample_grp == "treatment"]])

## 1. Hypothesis testing

In [ ]:
# Hypothesis testing.
import scipy.stats as stats
import numpy as np

degrade_df = temporal_adjust(afc_df)

dist_control = degrade_df[degrade_df.sample_grp == "control"].t2_agbd_a0 - degrade_df[degrade_df.sample_grp == "control"].t1_agbd_a0
dist_treatment = degrade_df[degrade_df.sample_grp == "treatment"].t2_agbd_a0 - degrade_df[degrade_df.sample_grp == "treatment"].t1_agbd_a0

# Plot distribution of S2 - S1 for control and treatment groups.
fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(16,8))
axi = ax[0]
axi.hist(dist_control, bins=50)

axi = ax[1]
axi.hist(dist_treatment, bins=50)

# Welch's t-test. Can we reject the null hypotheses that 
# (1) the mean of the treatment group is equal to the mean of the control group?
res = stats.ttest_ind(dist_control, dist_treatment, equal_var=False)
print(res.pvalue)
# (2) the mean of the control group is greater than the mean of the treatment group?
res = stats.ttest_ind(dist_control, dist_treatment, equal_var=False, alternative="greater")
print(res.pvalue)

# Does it also hold if we look only at single-pixel disturbances?
dist_treatment = degrade_df[(degrade_df.sample_grp == "treatment") & (degrade_df.measured_disturbance == 1)].t2_agbd_a0 - degrade_df[(degrade_df.sample_grp == "treatment") & (degrade_df.measured_disturbance == 1)].t1_agbd_a0

# Welch's t-test
# (1) the mean of the treatment group is equal to the mean of the control group?
res = stats.ttest_ind(dist_control, dist_treatment, equal_var=False)
print(res.pvalue)
# (2) the mean of the treatment group is greater than the mean of the control group?
res = stats.ttest_ind(dist_control, dist_treatment, equal_var=False, alternative="greater")
print(res.pvalue)

## 2. Biomass loss depends on ...

In [ ]:
afc_df = afc_df[afc_df.t1_agbd_a0 != 0]
glad_df = glad_df[glad_df.t1_agbd_a0 != 0]
afc_df["agbd_pct_loss"] = (afc_df.t1_agbd_a0 - afc_df.t2_agbd_a0) / (afc_df.t1_agbd_a0) * 100
glad_df["agbd_pct_loss"] = (glad_df.t1_agbd_a0 - glad_df.t2_agbd_a0) / (glad_df.t1_agbd_a0) * 100

#### ... time

In [ ]:
import scipy.stats as stats
# 0. Days since disturbance for disturbed samples
treatment_df = glad_df[glad_df.sample_grp == "treatment"]
disturb_days = np.array([treatment_df.p1_disturb_date, treatment_df.p2_disturb_date, treatment_df.p3_disturb_date, treatment_df.p4_disturb_date, treatment_df.p5_disturb_date, treatment_df.p6_disturb_date, treatment_df.p7_disturb_date, treatment_df.p8_disturb_date, treatment_df.p9_disturb_date]).T.astype(float)
disturb_days[disturb_days == 0] = np.nan
mode_disturb_days = stats.mode(disturb_days, axis=1, nan_policy='omit', keepdims=True)[0].squeeze()
treatment_df["mode_disturb_days"] = mode_disturb_days

In [ ]:
treatment_df["days_since_disturbance"] = treatment_df.t2_days - treatment_df.mode_disturb_days
# At the moment, because of the way we record the disturbance date, some of the days_since_disturbance
# values are negative, even in the treatment set.
# This is because the control disturb_date is also recorded whenever it is applicable
# (and AFTER the treatment disturb_date in the code, so it will override the treatment disturb_date).
# Many treatment disturbance pairs also have unmeasured disturbances at another time.
# For now, only include disturbances with a positive number of days since disturbance.
days_since_disturb_df = treatment_df[treatment_df.days_since_disturbance >= 0]
days_since_disturb_df["months_since_disturbance"] = days_since_disturb_df.days_since_disturbance // 30


#### ... intensity

In [ ]:
degrade_sdf = spark.read.parquet(
    (constants.RESULTS_PATH / "gedi_degradation_afc_intensity").as_posix()
)

intensity_df = gpd.GeoDataFrame(
    degrade_sdf.toPandas(), geometry="t2_geom"
).copy()
intensity_df = intensity_df[intensity_df["t1_agbd_a0"] != 0]
intensity_df.loc[intensity_df.control_disturbance > 0, "sample_grp"] = "control"
# Note: points may have a control disturbance as well as a measured disturbance.
# in that case, we include them in the treatment group; we don't care that they
# were also disturbed at another, unmeasured time.
intensity_df.loc[
    intensity_df.measured_disturbance > 0, "sample_grp"
] = "treatment"
print(len(intensity_df))
print(len(intensity_df[intensity_df["sample_grp"] == "treatment"]))
print(len(intensity_df[intensity_df["sample_grp"] == "control"]))

In [ ]:
intensity_df = intensity_df[(intensity_df.sample_grp == "treatment")]
# intensity_df = remove_duplicates(intensity_df)
# print(len(intensity_df))
intensity_df["agbd_pct_loss"] = (intensity_df.t1_agbd_a0 - intensity_df.t2_agbd_a0) / (intensity_df.t1_agbd_a0) * 100
intensity_df.disturb_intensity_old = intensity_df.disturb_intensity.copy()
intensity_df.disturb_intensity = intensity_df.disturb_intensity * 4 / intensity_df.measured_disturbance

bins = np.concatenate([np.arange(0, 0.1, 0.025), np.arange(0.1,0.71, 0.1)])
print(bins)
intensity_df["disturb_bin"] = pd.cut(intensity_df.disturb_intensity, bins=bins, labels=bins[:-1])
print(intensity_df[['disturb_intensity', 'disturb_bin']])
print(intensity_df.disturb_bin.value_counts())

### Now bootstrap and plots

In [ ]:
def bootstrap_and_plot_extent(degrade_df, axi):
    treatment_df = degrade_df[degrade_df.sample_grp == "treatment"]
    control_df = degrade_df[degrade_df.sample_grp == "control"]
    # Bootstrap confidence intervals for the median percent AGBD change.
    k_dist = np.max(degrade_df.measured_disturbance)
    # 1. Bootstrap sample the median pct delta AGBD for each disturbance extent and the control.
    n = 100
    medians = np.zeros((n, k_dist+1))
    for i in range(n):
        medians[i, 0] = np.median(control_df.agbd_pct_loss.sample(frac=1, replace=True))
        for j in range(k_dist):
            medians[i, j+1] = np.median(treatment_df[treatment_df.measured_disturbance == j+1].agbd_pct_loss.sample(frac=1, replace=True))
    print(medians.mean(axis=0))

    # 2. Compute the 95% confidence interval for each median for plotting.
    ci = np.zeros((2, k_dist+1))
    for i in range(k_dist+1):
        ci[:, i] = np.percentile(medians[:, i], [2.5, 97.5])
    print(ci)

    # 3. Compute a regression line for disturbance extent vs. median pct delta AGBD.
    intercepts = np.zeros(n)
    slopes = np.zeros(n)
    rs = np.zeros(n)
    pvals = np.zeros(n)
    for i in range(n):
        model = sm.OLS(medians[i, 1:k_dist+1], sm.add_constant(np.arange(1, k_dist+1)))
        results = model.fit()
        intercepts[i] = results.params[0]
        slopes[i] = results.params[1]
        rs[i] = results.rsquared
        pvals[i] = results.pvalues[1]

    # 4. Plot the distribution of median pct delta AGBD for each disturbance extent.
    axi.errorbar(np.arange(0, k_dist+1), medians.mean(axis=0), yerr=np.abs(ci - medians.mean(axis=0)), fmt='o', ecolor="tab:blue", capsize=3, zorder=1)
    axi.scatter(0, medians.mean(axis=0)[0], color="tab:green", zorder=10)
    axi.scatter(np.arange(1, k_dist+1), medians.mean(axis=0)[1:k_dist+1], color="tab:orange", zorder=10)
    # Plot a sample of the regression lines
    sample = np.random.choice(np.arange(n), size=25)
    for i in sample:
        axi.plot(np.arange(1, k_dist+1), intercepts[i] + slopes[i] * np.arange(1, k_dist+1), color="gray", alpha=0.2, zorder=5)
    axi.plot(np.arange(1, k_dist+1), intercepts.mean() + slopes.mean() * np.arange(1, k_dist+1), color="black", zorder=6)

    axi.set_xticks(np.arange(0, k_dist+1))
    axi.set_xlabel(f"Disturbance extent (1-{k_dist})")
    axi.set_ylabel("Median percent AGBD loss")
    axi.set_title(f"Biomass loss by disturbance extent (1-{k_dist})")

    legend_elements = [
            Line2D([0], [0], marker='o', color='tab:orange', label="Treatment",
                markerfacecolor='tab:orange', markersize=7, linestyle='None'),
            Line2D([0], [0], marker='o', color='tab:green', label="Control",
                markerfacecolor='tab:green', markersize=7, linestyle='None'),
            Line2D([0], [0], color='black', linewidth=3, label=f"y = {slopes.mean():.1f}x + {intercepts.mean():.1f}\n$R^2$ = {rs.mean():.2f}, p = {pvals.mean():.3f}"),
        ]
    axi.legend(loc="lower right", handles=legend_elements)
    print(rs.mean())
    print(pvals.mean())

In [ ]:
def bootstrap_and_plot_time(days_since_disturb_df, axi):
    k_months = 23 # 0-22 months since disturbance. Beyond 22 months we don't have enough data.
    # 1. Bootstrap sample the median pct delta AGBD for each number of months since disturbance.
    n = 100

    # with pd.option_context('display.max_rows', None, 'display.max_columns', None):
        # print(days_since_disturb_df.months_since_disturbance.value_counts().sort_index())
    medians = np.zeros((n, k_months))
    for i in range(n):
        for j in range(k_months):
            medians[i, j] = np.median(days_since_disturb_df[days_since_disturb_df.months_since_disturbance == j].agbd_pct_loss.sample(frac=1, replace=True))

    # 2. Compute the 95% confidence interval for each median for plotting.
    ci = np.zeros((2, k_months))
    for i in range(k_months):
        ci[:, i] = np.percentile(medians[:, i], [2.5, 97.5])

    # 3. Plot the distribution of median pct delta AGBD for each time since disturbance.

    axi.errorbar(np.arange(k_months), medians.mean(axis=0), yerr=np.abs(ci - medians.mean(axis=0)), fmt='o', ecolor="tab:blue", capsize=3, zorder=1)
    axi.scatter(0, medians.mean(axis=0)[0], color="tab:green", zorder=10)
    axi.scatter(np.arange(k_months), medians.mean(axis=0), color="tab:orange", zorder=10)

    # Try fitting a curve ...
    # intercept, linear, quadratic
    coeffs = np.zeros((n, 3))
    adjrs = np.zeros(n)
    # intercept, linear, quadratic
    ps = np.zeros((n, 3))
    for i in range(n):
        data = pd.DataFrame({'y': medians[i, :], 'x': np.arange(k_months)})
        model = smf.ols(formula = 'y ~ np.power(x, 2) + x', data=data)
        # model = smf.ols(formula = 'y ~ x', data=data)
        results = model.fit()
        coeffs[i, :] = results.params
        adjrs[i] = results.rsquared_adj
        ps[i] = results.pvalues[:3]

    print(coeffs.mean(axis=0))
    sample = np.random.choice(np.arange(n), size=25)
    for i in sample:
        axi.plot(np.arange(k_months), coeffs[i, 0] + coeffs[i, 1] * np.power(np.arange(k_months), 2) + coeffs[i, 2] * np.arange(k_months), color="gray", alpha=0.2, zorder=5)
    axi.plot(np.arange(k_months), coeffs.mean(axis=0)[0] + coeffs.mean(axis=0)[1] * np.power(np.arange(k_months), 2) + coeffs.mean(axis=0)[2] * np.arange(k_months), color="black", linewidth=3, zorder=6)

    axi.set_xticks(np.arange(0, k_months, 3))
    axi.set_xlabel(f"Months (rounded) since disturbance")
    axi.set_ylabel("Median percent AGBD loss")
    axi.set_ylim(0, None)
    axi.set_title(f"Biomass loss by time since disturbance")

    legend_elements = [
            Line2D([0], [0], marker='o', color='tab:orange', label="Treatment",
                markerfacecolor='tab:orange', markersize=7, linestyle='None'),
            Line2D([0], [0], color='black', linewidth=3, label=f"y = {coeffs.mean(axis=0)[0]:.1f} + {coeffs.mean(axis=0)[1]:.1f}x + {coeffs.mean(axis=0)[2]:.1f}$x^2$\nadj-$R^2$ = {adjrs.mean():.2f}"),
        ]
    axi.legend(loc="lower right", handles=legend_elements)
    print(adjrs.mean())
    print(ps.mean(axis=0))

In [ ]:
def bootstrap_and_plot_intensity(degrade_df, bins, axi):
    import statsmodels.formula.api as smf

    # Bootstrap confidence intervals for the median percent AGBD change.
    k_dist = len(bins) - 1
    bin_vals = bins[:-1]

    # 1. Bootstrap sample the median pct delta AGBD for each disturbance extent and the control.
    n = 100
    medians = np.zeros((n, k_dist))
    for i in range(n):
        for j, b in enumerate(bin_vals):
            medians[i, j] = np.median(degrade_df[degrade_df.disturb_bin == b].agbd_pct_loss.sample(frac=1, replace=True))
    print(medians.mean(axis=0))

    # 2. Compute the 95% confidence interval for each median for plotting.
    ci = np.zeros((2, k_dist))
    for i in range(k_dist):
        ci[:, i] = np.percentile(medians[:, i], [2.5, 97.5])
    print(ci)

    # 3. Compute a regression line for disturbance extent vs. median pct delta AGBD.
    intercepts = np.zeros(n)
    slopes = np.zeros(n)
    rs = np.zeros(n)
    pvals = np.zeros(n)

    def fun(a, b, xs):
        return a + b * np.sqrt(xs)

    for i in range(n):
        df = pd.DataFrame({'intensity': bin_vals, 'agbd_pct_loss': medians[i,:]})
        model = smf.ols('agbd_pct_loss ~ np.sqrt(intensity)', df)
        results = model.fit()
        intercepts[i] = results.params[0]
        slopes[i] = results.params[1]
        rs[i] = results.rsquared
        pvals[i] = results.pvalues[1]

    # 4. Plot the distribution of median pct delta AGBD for each disturbance extent.
    axi.errorbar(bin_vals, medians.mean(axis=0), yerr=np.abs(ci - medians.mean(axis=0)), fmt='o', ecolor="tab:blue", capsize=3, zorder=1)
    axi.scatter(bin_vals, medians.mean(axis=0), color="tab:orange", zorder=10)
    # Plot a sample of the regression lines
    sample = np.random.choice(np.arange(n), size=25)
    for i in sample:
        axi.plot(bin_vals, fun(intercepts[i], slopes[i], bin_vals), color="gray", alpha=0.2, zorder=5)
    axi.plot(bin_vals, fun(intercepts.mean(), slopes.mean(), bin_vals), color="black", zorder=6)
    print(bin_vals)

    labels = pd.cut(degrade_df.disturb_intensity * 4, bins=bins).cat.categories
    axi.set_xticks(bin_vals[[0,2,4,5,6,7,8]], labels=labels.astype(str)[[0,2,4,5,6,7,8]], rotation=-45)
    axi.set_xlabel(f"Disturbance intensity index")
    axi.set_ylabel("Median percent AGBD loss")
    axi.set_title(f"Biomass loss by disturbance intensity")

    legend_elements = [
            Line2D([0], [0], marker='o', color='tab:orange', label="Treatment",
                markerfacecolor='tab:orange', markersize=7, linestyle='None'),
            Line2D([0], [0], color='black', linewidth=3, label=f"y = {slopes.mean():.1f} * sqrt(x) + {intercepts.mean():.1f}\n$R^2$ = {rs.mean():.2f}, p = {pvals.mean():.4f}"),
        ]
    axi.legend(bbox_to_anchor=(1.0, 0.215), handles=legend_elements)
    print(rs.mean())
    print(pvals.mean())

In [ ]:
fig, ax = plt.subplots(nrows=2, ncols=2, figsize=(10, 10))
################################################################################
# A. GLAD disturbance extent
################################################################################
axi = ax[0][0]
bootstrap_and_plot_extent(glad_df, axi)

################################################################################
# B. AFC disturbance extent
################################################################################
axi = ax[0][1]
bootstrap_and_plot_extent(afc_df, axi)

################################################################################
# C. GLAD time since disturbance
################################################################################
axi = ax[1][0]
bootstrap_and_plot_time(days_since_disturb_df, axi)

################################################################################
# D. AFC disturbance intensity
################################################################################
axi = ax[1][1]
bootstrap_and_plot_intensity(intensity_df, bins, axi)

plt.tight_layout()

In [ ]:
# Check that there is still statistically significant positive correlation
# even if the disturbance extent is constant.
from scipy.stats import pearsonr
bins = np.concatenate([[0.0, 0.05], np.arange(0.1,0.61, 0.1)])
print(bins)
intensity_df["disturb_bin"] = pd.cut(intensity_df.disturb_intensity, bins=bins, labels=bins[:-1])
k_dist = len(bins) - 1
k_extent = np.max(intensity_df.measured_disturbance)
prs = []
for extent in range(1,k_extent+1):
    tdf = intensity_df[intensity_df.measured_disturbance == extent]
    print(tdf.disturb_bin.value_counts())
    n = 100
    medians = np.zeros((n, k_dist))
    for i in range(n):
        for j, b in enumerate(bins[:-1]):
            medians[i, j] = np.median(tdf[tdf.disturb_bin == b].agbd_pct_loss.sample(frac=1, replace=True))
    print(medians.mean(axis=0))
    prs.append(pearsonr(bins[:-1], medians.mean(axis=0)))

for pr in prs:
    print(pr)
